In [1]:
from dataset_analysis import *
from XMLAnnotParser import parse_single_annotation_file

## Test

In [2]:
annotations_top_dir = './annotation'
# dataset_dict_list = get_all_dicts(annotations_top_dir)

In [3]:
annotation_path = './annotation/h30_5_10/DJI_0002.xml'

In [4]:
xml_dict = parse_single_annotation_file(xml_path=annotation_path)

In [5]:
count_tracks(xml_dict)

'19'

In [6]:
count_categories(xml_dict)

{'person': 19}

In [7]:
count_attribute_occurrence(xml_dict)

{'65': 12001, '67': 12001, '69': 7458, '71': 12001, '73': 12001, '68': 4543}

In [8]:
count_frames_per_track(xml_dict)

{'0': 947,
 '1': 947,
 '2': 947,
 '3': 947,
 '4': 947,
 '5': 947,
 '6': 947,
 '7': 947,
 '8': 947,
 '9': 947,
 '10': 947,
 '11': 947,
 '12': 947,
 '13': 947,
 '14': 947,
 '15': 947,
 '16': 947,
 '17': 947,
 '18': 947}

In [9]:
count_attribute_occurrence_frame_level(xml_dict)

{'65': 11364947,
 '67': 11364947,
 '69': 7062726,
 '71': 11364947,
 '73': 11364947,
 '68': 4302221}

## Category distribution

In [ ]:
annotations_top_dir = './annotation'
total_category_distribution_dict = {}
total_num_of_categories = 0

In [ ]:
for annotations_path in os.listdir(annotations_top_dir):
    annotations_path = os.path.join(annotations_top_dir, annotations_path)
    for annotation_filename in os.listdir(annotations_path):
        if not annotation_filename.endswith(".xml"):
            continue
        # parse annotation file
        xml_path = os.path.join(annotations_path, annotation_filename)
        xml_dict = parse_single_annotation_file(xml_path)

        total_num_of_categories += count_categories(xml_dict)
        category_count_dict = count_categories(xml_dict)
